In [ ]:
%%time

from utils.utils import create_dataset, Trainer
from models.model import *
# from layer.layer import Embedding, FeaturesEmbedding, EmbeddingsInteraction, MultiLayerPerceptron

import math
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ConstantLR

from torch.utils.data import TensorDataset, DataLoader

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Training on [{}].'.format(device))

In [ ]:
%%time

# Load data and split train, validation, test sets.
SAMPLE_NUM = 10000
task = 'regression'  # 'classification'
dataset = create_dataset('movielens', sample_num=SAMPLE_NUM, task=task, device=device)
field_dims, (train_X, train_y), (valid_X, valid_y), (test_X, test_y) = dataset.train_valid_test_split()

In [ ]:
config = dict(
    EMBEDDING_DIM = 50,
    LEARNING_RATE = 0.25, #1e-4
    REGULARIZATION = 1e-6,
    BATCH_SIZE = 1024,  # 64
    EPOCH = 1200,
    TRIAL = 200)

In [ ]:
def train(model, criterion, config, 
          scheduler=None, draw=False, **kwargs):

    EMBEDDING_DIM = config.get('EMBEDDING_DIM')
    LEARNING_RATE = config.get('LEARNING_RATE')
    REGULARIZATION = config.get('REGULARIZATION')
    BATCH_SIZE = config.get('BATCH_SIZE')
    EPOCH = config.get('EPOCH')
    TRIAL = config.get('TRIAL')

    models = dict(
        MF = MatrixFactorization,
        EE = EuclideanEmbedding,
        BR = BetaRecommendation
    )

    model = models.get(model)(field_dims, EMBEDDING_DIM, **kwargs).to(device)

    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=REGULARIZATION)

    if scheduler == 'const':
        scheduler = ConstantLR(optimizer, factor=0.2, total_iters=300)
        
    trainer = Trainer(model, optimizer, criterion, BATCH_SIZE, scheduler=scheduler, task=task)
    train_loss, valid_loss = trainer.train(train_X, train_y, 
                                           valid_X=valid_X, valid_y=valid_y, 
                                           epoch=EPOCH, trials=TRIAL, draw=draw)
    test_loss, test_metric = trainer.test(test_X, test_y)
    print('test_loss:  {:.5f} | test_metric:  {:.5f}'.format(test_loss, test_metric))
    return train_loss, valid_loss, test_loss